# Object Creation: Self-Report Stress Objects from Mood Events

This notebook demonstrates how to create stress self-report objects from mood events in the OCED data. Each stress self-report object represents a stress value reported in a mood event, and is linked to:
<ul>
<li>The mood event that reported the stress
<li>The day on which it occurred
<li>The user who reported it
<li>The nearest notification within a specified time period (if any exists)

## 1. Setup

In [1]:

import sys
sys.path.append('..')
from pathlib import Path
import pandas as pd
from datetime import timedelta
from src.oced.oced_data_query import OCEDDataQuery
from src.oced.time_objects import TimeObject
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import gc
import json
import pandas as pd
from pathlib import Path
from typing import Dict, Any
from src.oced.stress_objects import StressObjectManager

## 2. Load OCED Data

In [2]:
# Get OCED-mHealth Data from JSON File
dataQuery = OCEDDataQuery()  
oced_data_file = f"player_107631_oced_data_time_bouts_notifications.json"
data_dict = dataQuery.load_json(oced_data_file)

# Quick look at the data structure
print(f"Number of behavior events: {len(data_dict.get('behaviorEvents', []))}")
print(f"Number of objects: {len(data_dict.get('objects', []))}")

# Diagnostic code to inspect notification events
mood_events = [
    notif for notif in data_dict.get('behaviorEvents', [])
    if notif['behaviorEventType'] == 'mood'
]
print(len(mood_events))
mood_events

Number of behavior events: 498
Number of objects: 273
76


[{'id': '67b8d80e-fb50-4037-9c3a-8db93746fc89',
  'behaviorEventType': 'mood',
  'time': '2025-05-25T13:00:00.886000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 6},
   {'name': 'arousal', 'value': 6},
   {'name': 'stress', 'value': 4}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '8d47453a-6a5c-454d-806a-971b71a11d8f',
    'type': 'object',
    'qualifier': 'occurred_on'}],
  'event_type': 'behavior'},
 {'id': '02a64aa1-1d70-4d0d-bb40-c94a0e515f19',
  'behaviorEventType': 'mood',
  'time': '2025-05-25T09:55:34.238000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 3},
   {'name': 'arousal', 'value': 4},
   {'name': 'stress', 'value': 7}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '8d47453a-6a5c-454d-806a-971b71a11d8f',
    'type': 'object',
    'qualifier': 'occurred_on'}],
  'event_

## 3. Initialize Stress Object Manager
Create an instance of the StressObjectManager class and add the stress self-report object type to the data.

In [3]:
# Initialize the stress object manager
manager = StressObjectManager()

# Add stress self-report object type to the data
data = manager.create_stress_object_type(data_dict)

# Print object types to verify
print("Object types in data:")
for obj_type in data.get('objectTypes', []):
    print(f"- {obj_type['name']}")

Stress self-report object type added.
Object types in data:
- player
- intervention
- day
- week
- physical_activity_bout
- notification
- stress_self_report


## 4. Create Stress Self-Report Objects
Now we'll create stress self-report objects from the mood events. We'll use a 1-hour time window to look for nearby notifications.

In [4]:
# Get the user ID from the data
user_objects = [obj for obj in data.get('objects', []) if obj['type'] == 'player']
if not user_objects:
    raise ValueError("No user object found in the data")
user_id = user_objects[0]['id']
print(f"Using user ID: {user_id}")

# Create stress self-report objects with a 1-hour time window
time_period = timedelta(hours=5)
extended_data, stress_objects = manager.create_stress_objects(
    data=data,
    user_id=user_id,
    time_period=time_period
)

Using user ID: d360ac20-326e-4f6f-8113-baa47ef536ef

Found 76 mood events
Found 0 existing stress self-report objects


Processing mood events:   4%|▍         | 3/76 [00:00<00:03, 23.48it/s]

Creating new stress self-report object 6b8ca64c-c7ee-45d6-9fd9-d71c08268a1a for mood event
Creating new stress self-report object 8a2c8bcc-c002-46f7-bade-380eaa66b534 for mood event
Creating new stress self-report object 1ae982cc-abfc-49a7-919c-b596eb7e3b75 for mood event
Creating new stress self-report object 9f71d86f-0437-45a2-8723-9b43d7ec9f0c for mood event
Creating new stress self-report object 2871268d-e996-45d7-b540-c646f6fbdfc4 for mood event


Processing mood events:  12%|█▏        | 9/76 [00:00<00:02, 23.75it/s]

Creating new stress self-report object 3b25b9e0-dbd0-49f3-88d8-7d08d8f29f47 for mood event
Creating new stress self-report object 6e182458-099b-4203-8688-f4b518a600e2 for mood event
Creating new stress self-report object 0ac5fa72-9a3f-4043-a399-7da00c0ec5ca for mood event
Creating new stress self-report object 30d4fb9a-136e-4f76-94cb-d6dbc42bf678 for mood event
Creating new stress self-report object 03df08a0-ebdd-40e7-9eeb-554911b9f683 for mood event


...

Linking stress self-report object to nearby notification 9323c14f-db3d-481e-bcb7-2041890a60b7
Creating new stress self-report object 39e69b21-c220-4905-8193-39215d1b5acc for mood event
Linking stress self-report object to nearby notification 6579f79e-bb4d-4266-aa31-b16495ca546f
Creating new stress self-report object e6752494-34fb-4a3f-9292-651dbfde9f5b for mood event
Linking stress self-report object to nearby notification 13787a9e-7074-4042-b8b6-8de0d03c0201
Creating new stress self-report object 153d3854-4bb2-4b0c-ac62-d80d2096fc89 for mood event
Linking stress self-report object to nearby notification e76ae730-c7a0-4f15-ad48-be3c8cd9b359
Creating new stress self-report object c95ce83b-63b6-4004-b0b8-597e8a0d00ea for mood event
Linking stress self-report object to nearby notification c19eb43e-99a1-405c-bdea-7c59d55814ac
Creating new stress self-report object ac3408b7-fefa-43d8-a48e-295b2dbf69f1 for mood event
Linking stress self-report object to nearby notification b2741941-2266-4047

Processing mood events:  96%|█████████▌| 73/76 [00:02<00:00, 24.79it/s]

Creating new stress self-report object dbecd66e-542c-474b-b43f-2a7667cec57c for mood event
Linking stress self-report object to nearby notification cb7591b4-702d-495a-a0d8-05c1a1342c28
Creating new stress self-report object d2dd53d8-7805-4ea7-8486-b395930d912e for mood event
Linking stress self-report object to nearby notification 5942eda9-8e90-41a5-9f35-de380a378e27
Creating new stress self-report object 629aace8-3da5-43ab-a5e6-6c72cd8acd96 for mood event
Linking stress self-report object to nearby notification 1dd64309-939f-4e63-9980-da77f52f7685
Creating new stress self-report object e25bf620-b27a-445e-b883-2076e91f8435 for mood event
Linking stress self-report object to nearby notification 5d513f5c-3933-4a0d-adb7-6a205a4ab17e
Creating new stress self-report object 313475ea-8b13-42c1-9c1c-87f910ae305e for mood event
Linking stress self-report object to nearby notification 41f35f23-c690-493a-bd24-aa6f5d9f89d9
Creating new stress self-report object 052a911b-bf67-489c-b82b-51937ab2aab3

Processing mood events: 100%|██████████| 76/76 [00:03<00:00, 24.69it/s]

Linking stress self-report object to nearby notification 2b0c60e0-642b-4f61-b5ce-277bb308e01d

Processing complete:
- Created 76 new stress self-report objects
- Total stress self-report objects: 76


## 5. Analyze Created Objects
Let's analyze the created stress self-report objects and their relationships.

In [5]:
# Count stress self-report objects with notification relationships
stress_with_notifications = [
    obj for obj in stress_objects
    if any(rel['qualifier'] == 'follows_notification' for rel in obj.get('relationships', []))
]

print(f"Stress self-report objects analysis:")
print(f"- Total stress self-report objects: {len(stress_objects)}")
print(f"- Stress self-report objects with notification relationships: {len(stress_with_notifications)}")
print(f"- Percentage with notifications: {len(stress_with_notifications)/len(stress_objects)*100:.1f}%")

# Look at a sample stress self-report object
if stress_objects:
    sample = stress_objects[0]
    print("\nSample stress self-report object:")
    print(f"ID: {sample['id']}")
    print("Attributes:")
    for attr in sample['attributes']:
        print(f"- {attr['name']}: {attr['value']} (at {attr['time']})")
    print("\nRelationships:")
    for rel in sample.get('relationships', []):
        print(f"- {rel['qualifier']} -> {rel['id']}")

Stress self-report objects analysis:
- Total stress self-report objects: 76
- Stress self-report objects with notification relationships: 52
- Percentage with notifications: 68.4%

Sample stress self-report object:
ID: 6b8ca64c-c7ee-45d6-9fd9-d71c08268a1a
Attributes:
- stress_value: 6.0 (at 2025-03-20T15:39:28.698000)

Relationships:
- occurred_on -> f7397705-b95e-4127-9090-32e0297be74a
- reported_by -> d360ac20-326e-4f6f-8113-baa47ef536ef


## 6. Save Extended Data
Finally, let's save the extended data with the new stress self-report objects.

In [6]:
# Save the extended data
manager.save_extended_data("player_107631_oced_data_time_bouts_notifications_stress.json", extended_data)

print("\nData saved successfully!")


Saving extended data with:
- 76 stress self-report objects
- 76 mood events
- 349 total objects
- 498 total behavior events
Saved extended data to: c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\notebooks\..\data\transformed\player_107631_oced_data_time_bouts_notifications_stress.json

Data saved successfully!


In [7]:
# Count stress self-report objects with notification relationships
data_objects = [
    obj for obj in extended_data.get('objects', [])
    if obj['type'] == 'stress_self_report'
]

print(len(data_objects))
data_objects

76


[{'id': '6b8ca64c-c7ee-45d6-9fd9-d71c08268a1a',
  'type': 'stress_self_report',
  'attributes': [{'name': 'stress_value',
    'value': 6.0,
    'time': '2025-03-20T15:39:28.698000'}],
  'relationships': [{'id': 'f7397705-b95e-4127-9090-32e0297be74a',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'reported_by'}]},
 {'id': '8a2c8bcc-c002-46f7-bade-380eaa66b534',
  'type': 'stress_self_report',
  'attributes': [{'name': 'stress_value',
    'value': 5.0,
    'time': '2025-03-21T11:12:32.086000'}],
  'relationships': [{'id': '109925f4-92f9-4171-8de8-c158ce3c4bb9',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'reported_by'}]},
 {'id': '1ae982cc-abfc-49a7-919c-b596eb7e3b75',
  'type': 'stress_self_report',
  'attributes': [{'name': 'stress_value',
    'value': 5.0,
    'time': '2025-03-24T12:16:29.4

In [8]:
# Count stress self-report objects with notification relationships
data = [
    obj for obj in extended_data.get('objectTypes', [])
]

print(len(data))
data

7


[{'name': 'player', 'attributes': [{'name': 'id', 'type': 'string'}]},
 {'name': 'intervention',
  'attributes': [{'name': 'goal', 'type': 'string'},
   {'name': 'start_date', 'type': 'string'},
   {'name': 'end_date', 'type': 'string'}]},
 {'name': 'day',
  'attributes': [{'name': 'date', 'type': 'string'},
   {'name': 'day_of_week', 'type': 'string'}]},
 {'name': 'week',
  'attributes': [{'name': 'week_start_date', 'type': 'string'},
   {'name': 'week_number', 'type': 'integer'},
   {'name': 'year', 'type': 'integer'}]},
 {'name': 'physical_activity_bout',
  'attributes': [{'name': 'bout_type', 'type': 'string'}]},
 {'name': 'notification',
  'attributes': [{'name': 'last_action', 'type': 'string'}]},
 {'name': 'stress_self_report',
  'attributes': [{'name': 'stress_value', 'type': 'number'}]}]

In [9]:
# Count stress self-report objects with notification relationships
data = [
    obj for obj in extended_data.get('behaviorEvents', [])
    if obj['behaviorEventType'] == 'mood'
]

print(len(data))
data

76


[{'id': '67b8d80e-fb50-4037-9c3a-8db93746fc89',
  'behaviorEventType': 'mood',
  'time': '2025-05-25T13:00:00.886000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 6},
   {'name': 'arousal', 'value': 6},
   {'name': 'stress', 'value': 4}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '8d47453a-6a5c-454d-806a-971b71a11d8f',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': '052a911b-bf67-489c-b82b-51937ab2aab3',
    'type': 'object',
    'qualifier': 'reports_stress'}],
  'event_type': 'behavior'},
 {'id': '02a64aa1-1d70-4d0d-bb40-c94a0e515f19',
  'behaviorEventType': 'mood',
  'time': '2025-05-25T09:55:34.238000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 3},
   {'name': 'arousal', 'value': 4},
   {'name': 'stress', 'value': 7}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id